# Overview of classical NLP techniques for text classification using Python libraries such as datasets, sklearn, and nltk.

In [1]:
%%capture
!pip install datasets fasttext "numpy<2" ftfy nltk pymorphy3 pymorphy3-dicts-uk optuna # for fasttext

Add a HF_TOKEN to run this notebook (key on the left)

### We would use a ukrainian news dataset for clasification task, consisting of approximately 61K news articles.

In [2]:
import datasets

dataset = datasets.load_dataset("shamotskyi/ukr_pravda_2y", split="train")
dataset

Dataset({
    features: ['art_id', 'date_published', 'tags', 'ukr_uri', 'ukr_title', 'ukr_author_name', 'ukr_text', 'ukr_tags', 'ukr_tags_full', 'rus_uri', 'rus_title', 'rus_author_name', 'rus_text', 'rus_tags', 'rus_tags_full', 'eng_uri', 'eng_title', 'eng_author_name', 'eng_text', 'eng_tags', 'eng_tags_full'],
    num_rows: 61629
})

### Sanity check for the data

In [3]:
dataset[0]

{'art_id': 7411594,
 'date_published': '2023-07-16',
 'tags': 'trivoga',
 'ukr_uri': 'https://www.pravda.com.ua/news/2023/07/16/7411594/',
 'ukr_title': 'У Києві та низці областей на 10 хвилин оголошували повітряну тривогу',
 'ukr_author_name': 'Катерина Тищенко',
 'ukr_text': 'У Києві та ще низці регіонів України оголошували повітряну тривогу через загрозу застосування балістичного озброєння. Джерело: alerts.in.ua, Повітряні сили Деталі: Близько 18:52 у столиці та інших регіонах оголосили повітряну тривогу. Пізніше Повітряні сили повідомили, що в Чернігівській, Київській, Сумській, Полтавській, Черкаській, Дніпропетровській, Донецькій, Запорізькій областях існує загроза застосування балістичного озброєння. Однак вже о 19:02 тривогу скасували майже в усіх областях.',
 'ukr_tags': 'повітряна тривога',
 'ukr_tags_full': "[('trivoga', 'повітряна тривога', '/tags/trivoga/')]",
 'rus_uri': 'https://www.pravda.com.ua/rus/news/2023/07/16/7411594/',
 'rus_title': 'В Киеве и ряде областей на 10

In [4]:
dataset = dataset.select_columns(["ukr_text", "ukr_tags", "tags", "ukr_tags_full"])
dataset[0]

{'ukr_text': 'У Києві та ще низці регіонів України оголошували повітряну тривогу через загрозу застосування балістичного озброєння. Джерело: alerts.in.ua, Повітряні сили Деталі: Близько 18:52 у столиці та інших регіонах оголосили повітряну тривогу. Пізніше Повітряні сили повідомили, що в Чернігівській, Київській, Сумській, Полтавській, Черкаській, Дніпропетровській, Донецькій, Запорізькій областях існує загроза застосування балістичного озброєння. Однак вже о 19:02 тривогу скасували майже в усіх областях.',
 'ukr_tags': 'повітряна тривога',
 'tags': 'trivoga',
 'ukr_tags_full': "[('trivoga', 'повітряна тривога', '/tags/trivoga/')]"}

In [5]:
dataset[4]

{'ukr_text': '5 грудня, у річницю підписання Будапештського меморандуму, російські війська влучили в енергетичні об’єкти у Київській, Вінницькій та Одеській областях України. Джерело: прем\'єр-міністр України Денис Шмигаль Пряма мова: "Терористична країна Росія спробувала знову реалізувати свій злочинний план — занурити Україну в темряву та холод. Завдяки героїчним ЗСУ та силам ППО ворогу вкотре не вдалося задумане. Енергетична система країни функціонує та залишається цілісною. Були влучання в енергоооб’єкти у Київській, Вінницькій, Одеській областях. У деяких регіонах вимушено застосовано екстрені відключення для збалансування системи й уникнення аварій. Рятувальники вже працюють над ліквідацією наслідків атаки, аби повернути світло в кожну домівку". Деталі: Шмигаль зазначив, що черговий масований ракетний удар Росія завдала в річницю підписання Будапештського меморандуму. "Демонструють світові, який із них "надійний" гарант безпеки. Тож покладаємося на Збройні Сили України та наших р

In [6]:
# first basic filter
dataset = dataset.filter(lambda x: isinstance(x["ukr_text"], str))
dataset = dataset.filter(lambda x: isinstance(x["ukr_tags"], str))
dataset

Dataset({
    features: ['ukr_text', 'ukr_tags', 'tags', 'ukr_tags_full'],
    num_rows: 56131
})

In [7]:
dataset = dataset.map(lambda x: {'ukr_tags': [i.strip() for i in x['ukr_tags'].split(',')]}, num_proc=8)
dataset[4]

{'ukr_text': '5 грудня, у річницю підписання Будапештського меморандуму, російські війська влучили в енергетичні об’єкти у Київській, Вінницькій та Одеській областях України. Джерело: прем\'єр-міністр України Денис Шмигаль Пряма мова: "Терористична країна Росія спробувала знову реалізувати свій злочинний план — занурити Україну в темряву та холод. Завдяки героїчним ЗСУ та силам ППО ворогу вкотре не вдалося задумане. Енергетична система країни функціонує та залишається цілісною. Були влучання в енергоооб’єкти у Київській, Вінницькій, Одеській областях. У деяких регіонах вимушено застосовано екстрені відключення для збалансування системи й уникнення аварій. Рятувальники вже працюють над ліквідацією наслідків атаки, аби повернути світло в кожну домівку". Деталі: Шмигаль зазначив, що черговий масований ракетний удар Росія завдала в річницю підписання Будапештського меморандуму. "Демонструють світові, який із них "надійний" гарант безпеки. Тож покладаємося на Збройні Сили України та наших р

In [8]:
count_tags = {}
for tags in dataset['ukr_tags']:
    for tag in tags:
        if tag not in count_tags:
            count_tags[tag] = 0
        count_tags[tag] += 1

count_tags = dict(sorted(count_tags.items(), key=lambda item: item[1], reverse=True))
print("Number of unique tags:", len(count_tags))
count_tags

Number of unique tags: 984


{'війна': 24880,
 'Росія': 16277,
 'Україна': 5899,
 'Зеленський': 3832,
 'США': 3744,
 'окупація': 2925,
 'зброя': 2637,
 'Збройні сили': 2510,
 'Путін': 2155,
 'обстріл': 2127,
 'ЄС': 1906,
 'Київ': 1881,
 'Херсонська область': 1641,
 'Генштаб': 1609,
 'Донецька область': 1360,
 'Німеччина': 1338,
 'НАТО': 1338,
 'Білорусь': 1333,
 'СБУ': 1312,
 'санкції': 1295,
 'безпілотники': 1227,
 'Велика Британія': 1182,
 'Польща': 1114,
 'розвідка': 1112,
 'Харківська область': 1060,
 'Крим': 1050,
 'вибух': 1037,
 'Маріуполь': 964,
 'жертви': 954,
 'допомога Україні': 926,
 'Луганська область': 901,
 'Запорізька область': 867,
 'Херсон': 860,
 'Сумська область': 814,
 'Київська область': 803,
 'авіа': 802,
 'ракетний удар': 791,
 'пропаганда': 747,
 'повітряна тривога': 730,
 'ППО': 719,
 'Харків': 641,
 'Дніпропетровська область': 638,
 'суд': 638,
 'армія': 632,
 'Байден': 622,
 'евакуація': 621,
 'Бахмут': 614,
 'Міноборони': 609,
 'кордон': 595,
 'мобілізація': 591,
 'Верховна Рада': 587,

In [9]:
fasttext_lang_id_url = "https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin"
import fasttext
import os
if not os.path.exists("lid.176.bin"):
    import urllib.request
    urllib.request.urlretrieve(fasttext_lang_id_url, "lid.176.bin")
ft_lang_id = fasttext.load_model("lid.176.bin")

In [10]:
sentences_to_test = [
    "Хто тримає цей район?",
    "Who holds this neighborhood?",
    "Хто тримає цей район? Who holds this neighborhood?",
    "Who holds this neighborhood? Хто тримає цей район? ",
    "Who holds this neighborhood? Хто тримає цей район? Maybe some kind of a dog?"
]

for sentence in sentences_to_test:
    print(sentence)
    print(ft_lang_id.predict(sentence, k=3))
    print("=====")


Хто тримає цей район?
(('__label__uk', '__label__el', '__label__ar'), array([9.99939144e-01, 3.24452776e-05, 3.03752004e-05]))
=====
Who holds this neighborhood?
(('__label__en', '__label__hu', '__label__fr'), array([0.9793033 , 0.00456449, 0.00240685]))
=====
Хто тримає цей район? Who holds this neighborhood?
(('__label__uk', '__label__en', '__label__el'), array([0.86032069, 0.02375383, 0.01937906]))
=====
Who holds this neighborhood? Хто тримає цей район? 
(('__label__uk', '__label__en', '__label__el'), array([0.86032081, 0.0237538 , 0.01937906]))
=====
Who holds this neighborhood? Хто тримає цей район? Maybe some kind of a dog?
(('__label__uk', '__label__en', '__label__ru'), array([0.56096339, 0.22342798, 0.03072331]))
=====


In [11]:
print("Before filtering:", len(dataset))

wrong_language = dataset.filter(lambda x: ft_lang_id.predict(x["ukr_text"].replace("\n", " "))[0][0] != "__label__uk", num_proc=4)
wrong_language

Parameter 'function'=<function <lambda> at 0x1273b11c0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Before filtering: 56131


Filter (num_proc=4):   0%|          | 0/56131 [00:00<?, ? examples/s]

Dataset({
    features: ['ukr_text', 'ukr_tags', 'tags', 'ukr_tags_full'],
    num_rows: 4
})

In [12]:
from IPython.display import display
display(wrong_language[3])
display(wrong_language[2])

#print(wrong_language[2]["ukr_text"])

{'ukr_text': 'Президент України Володимир Зеленський закликав солдатів російського окупанта закінчити війну та повернутися додому, бо український народ стоятиме до кінця. Джерело: звернення глави держави від 9 березня Пряма мова (Зеленський говорив російською) "Российские солдаты! У вас ещё есть шанс выжить. Почти две недели нашего сопротивления показали вам, что мы не сдадимся. Что мы будем воевать, пока не вернём свою землю и пока не ответим сполна за всех наших убитых людей. За убитых детей. Вы ещё можете спастись. Если просто уйдёте. Не верьте своим командирам, когда они говорят вам, что у вас ещё есть шансы в Украине. Вас здесь ничего не ждёт. Кроме плена или гибели, вы забираете наши жизни и отдадите свои. И мы знаем – у нас есть перехваты – что ваши командиры уже все всё понимают. Нужно заканчивать эту войну. Нужно возвращаться к миру. Уходите из наших домов, уходите к себе".',
 'ukr_tags': ['війна', 'Зеленський'],
 'tags': 'vijna,zelensky',
 'ukr_tags_full': "[('vijna', 'війна'

{'ukr_text': 'Російський загарбник у телефонній розмові з дружиною похвалився, що накрав для неї з українського будинку "косметики, кросівки фірмові і якісних футболок", дружина окупанта попросила ноутбук і спортивний костюм. Джерело: СБУ, яка публікує перехоплені розмови російських загарбників Деталі: Окупант розповів, що мешканці дому, який він грабував, жили дуже гарно – ремонт у них красивий, речі всі якісні, вони пили вітаміни "дорогі", і навіть мали сауну, у якій окупанти "паряться вже другий день". Далі приводимо діалог окупанта Андрія з його дружиною: Загарбник Андрій: "Я тут маленько подукрал косметики вам немножко. Там пробники, правда". Дружина окупанта: "Ну и ладно – это привет из Украины будет. Че, нормально. Ну какой русский человек не сопрет ниче". Загарбник Андрій: "Кроссовки потом женские, ну они NB, они фирменные, тут все такое. 38 размер. Они вообще классные. Тут все качественное, вся одежда". Дружина окупанта: "Да там, наверное, все мальчишки набрали". Загарбник Анд

In [13]:
dataset = dataset.filter(lambda x: ft_lang_id.predict(x["ukr_text"].replace("\n", " "))[0][0] == "__label__uk", num_proc=4)
print("After filtering:", len(dataset))
dataset

Filter (num_proc=4):   0%|          | 0/56131 [00:00<?, ? examples/s]

After filtering: 56127


Dataset({
    features: ['ukr_text', 'ukr_tags', 'tags', 'ukr_tags_full'],
    num_rows: 56127
})

In [14]:
# replace with <email>, <url>, <phone>
import re

def replace_sensitive_info(text):
    # Email pattern
    email = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    # URL pattern
    url = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # Phone number pattern
    phone = r'(\+\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'

    text = re.sub(email, '<email>', text)
    text = re.sub(url, '<url>', text)
    text = re.sub(phone, '<phone>', text)

    return text
dataset = dataset.map(lambda x: {"ukr_text": replace_sensitive_info(x["ukr_text"])}, num_proc=8)
print(dataset.filter(lambda x: "<email>" in x["ukr_text"], num_proc=4)[0])

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/56127 [00:00<?, ? examples/s]

{'ukr_text': 'Мобілізація жителів окупованого Криму на війну проти України є воєнним злочином Російської Федерації, прокуратура АР Крим радить жителям півострова у випадку призову в російську армію інформувати її про це й не вчиняти злочинів проти України. Джерело: прокуратура Автономної Республіки Крим та міста Севастополь, її керівник Ігор Поночовний Пряма мова Поночовного: "Мобілізація, яку проводить РФ в окупованому Криму, – черговий воєнний злочини проти громадян України, що мешкають на півострові". Деталі: Керівник прокуратури АРК зазначив, що  російська влада з 2015 року незаконно примушувала кримчан служити в її збройних силах, щонайменше 34 тисячі осіб потрапили до ЗС РФ. А з 21 вересня 2022 року, наголосив Поночовний, "кримчан примушують помирати за ідеї "руського міра" у війні проти своїх же громадян". Відповідно до ст. 51 Женевської конвенції про захист цивільного населення під час війни від 1949 року, державі-окупанту заборонено примушувати цивільне населення окупованої те

In [15]:
# lowercase / uppercase?

dataset = dataset.map(lambda x: {"ukr_text": x["ukr_text"].lower()}, num_proc=8)

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

In [16]:
# check chars

def extract_chars(dataset):
    dataset = dataset.map(lambda x: {"chars": set(x["ukr_text"])}, num_proc=8)

    total_chars = set()
    for item in dataset:
        total_chars.update(item["chars"])
    return total_chars

total_chars = extract_chars(dataset)
total_chars


Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

{'\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '~',
 '£',
 '§',
 '«',
 '\xad',
 '°',
 '±',
 '·',
 '»',
 '¿',
 '×',
 'ß',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'ì',
 'í',
 'î',
 'ï',
 'ð',
 'ñ',
 'ó',
 'ô',
 'õ',
 'ö',
 'ø',
 'ù',
 'ú',
 'ü',
 'ý',
 'ā',
 'ă',
 'ą',
 'ć',
 'č',
 'ď',
 'đ',
 'ē',
 'ė',
 'ę',
 'ě',
 'ğ',
 'ī',
 'ı',
 'ķ',
 'ļ',
 'ľ',
 'ł',
 'ń',
 'ņ',
 'ň',
 'ő',
 'ř',
 'ś',
 'ş',
 'š',
 'ţ',
 'ť',
 'ū',
 'ů',
 'ű',
 'ų',
 'ź',
 'ż',
 'ž',
 'ț',
 'ə',
 'ʼ',
 '˗',
 'ˮ',
 '́',
 '̇',
 '̈',
 '̓',
 '̧',
 '̶',
 'ά',
 'έ',
 'ή',
 'ί',
 'α',
 'β',
 'ε',
 'η',
 'ι',
 'κ',
 'λ',
 'μ',
 'ν',
 'ο'

In [17]:
len(total_chars)

591

In [18]:
dataset.filter(lambda x: "🫡" in x["ukr_text"])[0]

Filter:   0%|          | 0/56127 [00:00<?, ? examples/s]

{'ukr_text': 'технологічний репортер new york times райан мак повідомив, що з twitter звільняються сотні співробітників. джерело: райан мак у twitter, bbc pink floyd\'s "wish you were here," employees signing off a video call while musk is pitching his vision, a wave of 🫡 emojis.inside decision day at twitter where hundreds of people may have just resigned.(w/ @mikeisaac, @dmccabe & @kateconger) <url> пряма мова райана мака: "день ухвалення рішень у twitter, коли сотні людей, можливо, щойно звільнилися". деталі: twitter повідомив співробітникам, що офісні будівлі компанії буде тимчасово закрито до 21 листопада. причину переїзду не вказано, зазначає bbc. оголошення з’явилося на тлі повідомлень про те, що велика кількість співробітників звільнилася. що було раніше: мільярдер та власник twitter ілон маск розіслав працівникам електронні листи, у яких вимагає зобов’язатись працювати у високо інтенсивному режимі або прийняти пропозицію звільнення з компанії з компенсацією.',
 'ukr_tags': ['М

In [19]:
dataset.filter(lambda x: "ß" in x["ukr_text"])[0]

Filter:   0%|          | 0/56127 [00:00<?, ? examples/s]

{'ukr_text': 'лідер єврейської громади австрії оскар дойч заявив, що у середу вночі на єврейській частині центрального кладовища відня сталася пожежа, а зовнішні стіни розмалювали свастикою. пожежна служба та поліція проводять розслідування. джерело: "європейська правда" з посиланням на ap деталі: за його словами, пожежа спалила вхідний вестибюль до церемоніальної зали, але ніхто не постраждав. за його словами, пожежна служба та поліція проводять розслідування. in der nacht wurde am jüdischen teil des zentralfriedhofs (iv. tor) ein brand gelegt. der vorraum der zeremonienhalle ist ausgebrannt. an außenmauern wurden hakenkreuze gesprayt. personen kamen nicht zu schaden. feuerwehr und polizei ermitteln. pic.twitter.com/llvcrrxige речник пожежної служби геральд шимпф заявив, що пожежа, схоже, сталася вночі, і до моменту виклику пожежників невдовзі після 8-ї ранку вогонь майже повністю згас. водночас канцлер країни карл негаммер заявив у twitter (x), що він засуджує напад на єврейське клад

In [20]:
import ftfy
def fix_text(text):
    return ftfy.fix_text(text)

dataset = dataset.map(lambda x: {"ukr_text": fix_text(x["ukr_text"])}, num_proc=8)

total_chars = extract_chars(dataset)

print("After fixing:", len(total_chars))
total_chars

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

After fixing: 584


{'\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '~',
 '£',
 '§',
 '«',
 '\xad',
 '°',
 '±',
 '·',
 '»',
 '¿',
 '×',
 'ß',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'ì',
 'í',
 'î',
 'ï',
 'ð',
 'ñ',
 'ó',
 'ô',
 'õ',
 'ö',
 'ø',
 'ù',
 'ú',
 'ü',
 'ý',
 'ā',
 'ă',
 'ą',
 'ć',
 'č',
 'ď',
 'đ',
 'ē',
 'ė',
 'ę',
 'ě',
 'ğ',
 'ī',
 'ı',
 'ķ',
 'ļ',
 'ľ',
 'ł',
 'ń',
 'ņ',
 'ň',
 'ő',
 'ř',
 'ś',
 'ş',
 'š',
 'ţ',
 'ť',
 'ū',
 'ů',
 'ű',
 'ų',
 'ź',
 'ż',
 'ž',
 'ț',
 'ə',
 '˗',
 'ˮ',
 '́',
 '̇',
 '̈',
 '̓',
 '̧',
 '̶',
 'ά',
 'έ',
 'ή',
 'ί',
 'α',
 'β',
 'ε',
 'η',
 'ι',
 'κ',
 'λ',
 'μ',
 'ν',
 'ο',
 'π',
 'ρ',

In [21]:
def basic_cleaning(text):
    text = text.replace("`", "'")
    text = text.replace("ʼ", "'")
    text = text.replace("…", "...")

    symbols = {
        "”": '"',
        "“": '"',
        "’": '"',
        "‘": '"',
        "«": '"',
        "»": '"',
        "–": "-",
        "—": "-",
        "―": "-",
    }
    for symbol, value in symbols.items():
        text = text.replace(symbol, value)
    return text

dataset = dataset.map(lambda x: {"ukr_text": basic_cleaning(x["ukr_text"])}, num_proc=8)
total_chars = extract_chars(dataset)
print("After basic cleaning:", len(total_chars))

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

After basic cleaning: 578


In [22]:
target_chats = set("абвгґдеєжзиіїйклмнопрстуфхцчшщьюя' ") # remove emails, urls, phones

def normalize_text(text):
    # leave only allowed characters
    # replace with a space, then remove extra spaces
    for char in text:
        if char.lower() not in target_chats:
            text = text.replace(char, " ")

    text = " ".join(text.split())
    return text
dataset = dataset.map(lambda x: {"ukr_text": normalize_text(x["ukr_text"])}, num_proc=8)
total_chars = extract_chars(dataset)
print("After normalization:", len(total_chars))

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

After normalization: 35


In [23]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Download required NLTK data
nltk.download('stopwords')

try:
    # Initialize Ukrainian stemmer and stopwords
    ukrainian_stopwords = set(stopwords.words('ukrainian'))
except:
    print("No ukrainian stopwords in nltk")


No ukrainian stopwords in nltk


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robinhad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
dataset[0]["ukr_text"]

'у києві та ще низці регіонів україни оголошували повітряну тривогу через загрозу застосування балістичного озброєння джерело повітряні сили деталі близько у столиці та інших регіонах оголосили повітряну тривогу пізніше повітряні сили повідомили що в чернігівській київській сумській полтавській черкаській дніпропетровській донецькій запорізькій областях існує загроза застосування балістичного озброєння однак вже о тривогу скасували майже в усіх областях'

In [25]:
# check words

def extract_words(dataset):
    dataset = dataset.map(lambda x: {"words": set(x["ukr_text"].split(" "))}, num_proc=8)

    total_words = set()
    for item in dataset:
        total_words.update(item["words"])
    return total_words

total_words = extract_words(dataset)

print("Total words:", len(total_words))
total_words

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

Total words: 185678


{'вставляти',
 'незадовільному',
 'ешелоновану',
 'непрацюючу',
 'санжарівки',
 'примітивна',
 'непроникними',
 'пропорційне',
 'маті',
 'авіаквиток',
 "безим'яне",
 'барменом',
 'наплічників',
 'блокуючі',
 'диких',
 'повтряну',
 'осколковій',
 'оберенчуком',
 'опалювального',
 'виношували',
 'регульованого',
 'строковики',
 'сторожити',
 'константа',
 'поло',
 'черемсі',
 'розглядаєте',
 'ділєрбека',
 'незаконному',
 'зеленому',
 'отримаєте',
 'демократи',
 'октокоптер',
 'атестат',
 'гарантируем',
 'морпіхам',
 'найактивнішого',
 'принцом',
 "кам'янчан",
 'диван',
 'попередження',
 'щитами',
 'маріс',
 'циркореферендуму',
 'комо',
 'гіффі',
 'довколо',
 'скандальності',
 'грейс',
 'опікунських',
 'янчук',
 'лухамаа',
 'стороженко',
 'мовники',
 'прикручуючи',
 'мачадо',
 'самостійного',
 'відриватися',
 'генеруватиме',
 'надію',
 'васильківського',
 'протекції',
 'французів',
 'волелюбність',
 'оцінці',
 'ярославській',
 'мікроавтобуса',
 'клаусом',
 'доріжкою',
 'ирії',
 'меморіалі

In [26]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer(lang='uk')

def ukrainian_stem(word):
    """Get the normal form (lemma) of a Ukrainian word"""
    parsed = morph.parse(word)[0]
    return parsed.normal_form

for test_words in ["працював", "бажаного", "робоча"]:
    print(test_words, "->", ukrainian_stem(test_words))


працював -> працювати
бажаного -> бажане
робоча -> робочий


In [27]:
dataset = dataset.map(lambda x: {"ukr_text": " ".join([ukrainian_stem(word) for word in x["ukr_text"].split(" ")])}, num_proc=8)
total_words = extract_words(dataset)
print("After stemming and stopwords removal:", len(total_words))

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/56127 [00:00<?, ? examples/s]

After stemming and stopwords removal: 78782


In [ ]:
dataset[0] # Місто Кий?

{'ukr_text': 'у кий та ще низка регіон україна оголошувати повітряний тривога через загроза застосування балістичний озброєння джерело повітряний сила деталь близько у столиця та інший регіон оголосити повітряний тривога пізніший повітряний сила повідомити що в чернігівський київський сумський полтавський черкаський дніпропетровський донецький запорізький область існувати загроза застосування балістичний озброєння однак вже о тривога скасувати майже в увесь область',
 'ukr_tags': ['повітряна тривога'],
 'tags': 'trivoga',
 'ukr_tags_full': "[('trivoga', 'повітряна тривога', '/tags/trivoga/')]"}

In [29]:
dataset = dataset.map(lambda x: {"target": 1 if "війна" in x["ukr_tags"] else 0})

target_counts = dataset['target'].count(1), dataset['target'].count(0)
print("Target counts (1, 0):", target_counts)

Map:   0%|          | 0/56127 [00:00<?, ? examples/s]

Target counts (1, 0): (24861, 31266)


In [30]:
# train test split
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_test

DatasetDict({
    train: Dataset({
        features: ['ukr_text', 'ukr_tags', 'tags', 'ukr_tags_full', 'target'],
        num_rows: 44901
    })
    test: Dataset({
        features: ['ukr_text', 'ukr_tags', 'tags', 'ukr_tags_full', 'target'],
        num_rows: 11226
    })
})

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Create document-term matrix using word counts
count_vectorizer = CountVectorizer(max_features=1000)
X_train = count_vectorizer.fit_transform(train_test['train']['ukr_text'])
X_test = count_vectorizer.transform(train_test['test']['ukr_text'])

# Get training and test labels
y_train = train_test['train']['target']
y_test = train_test['test']['target']

# Initialize and train the MultinomialNB classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Make predictions
y_pred = nb_classifier.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print model accuracy
print(f"Model accuracy: {nb_classifier.score(X_test, y_test):.3f}")

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.76      0.74      6296
           1       0.67      0.63      0.65      4930

    accuracy                           0.70     11226
   macro avg       0.70      0.70      0.70     11226
weighted avg       0.70      0.70      0.70     11226

Model accuracy: 0.705


In [32]:
import numpy as np
from sklearn.metrics import classification_report

# Create random predictions
np.random.seed(42)  # for reproducibility
y_pred_random = np.random.random(len(y_test)) >= 0.5  # random binary predictions
y_pred_random = y_pred_random.astype(int)

# Print the classification report for random baseline
print("Random Baseline Classification Report:")
print(classification_report(y_test, y_pred_random))

# Calculate accuracy for random baseline
random_accuracy = (y_test == y_pred_random).mean()
print(f"Random baseline accuracy: {random_accuracy:.3f}")

Random Baseline Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.51      0.54      6296
           1       0.45      0.50      0.47      4930

    accuracy                           0.51     11226
   macro avg       0.51      0.51      0.51     11226
weighted avg       0.51      0.51      0.51     11226

Random baseline accuracy: 0.508


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

# Create document-term matrix using word counts
count_vectorizer = CountVectorizer(max_features=1000)
X_train = count_vectorizer.fit_transform(train_test['train']['ukr_text'])
X_test = count_vectorizer.transform(train_test['test']['ukr_text'])

# Get training and test labels
y_train = train_test['train']['target']
y_test = train_test['test']['target']

# Initialize and train the RandomForest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, verbose=2)
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Print the classification report
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))

# Print model accuracy
print(f"Random Forest accuracy: {rf_classifier.score(X_test, y_test):.3f}")

# Print feature importance (optional)
feature_importance = list(zip(count_vectorizer.get_feature_names_out(), rf_classifier.feature_importances_))
sorted_features = sorted(feature_importance, key=lambda x: x[1], reverse=True)
print("\nTop 10 most important words:")
for word, importance in sorted_features[:10]:
    print(f"{word}: {importance:.4f}")

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   22.0s


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   53.6s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.83      0.80      6296
           1       0.76      0.68      0.71      4930

    accuracy                           0.76     11226
   macro avg       0.76      0.75      0.75     11226
weighted avg       0.76      0.76      0.76     11226



[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


Random Forest accuracy: 0.762

Top 10 most important words:
обстріл: 0.0138
російський: 0.0136
військо: 0.0127
окупант: 0.0126
європейський: 0.0110
ворог: 0.0082
військовий: 0.0079
втрата: 0.0076
росіянин: 0.0072
до: 0.0070


In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

# Create TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train = tfidf_vectorizer.fit_transform(train_test['train']['ukr_text'])
X_test = tfidf_vectorizer.transform(train_test['test']['ukr_text'])

# Get training and test labels
y_train = train_test['train']['target']
y_test = train_test['test']['target']

# Initialize and train the RandomForest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, verbose=2)
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Print the classification report
print("Random Forest with TF-IDF Classification Report:")
print(classification_report(y_test, y_pred))

# Print model accuracy
print(f"Random Forest with TF-IDF accuracy: {rf_classifier.score(X_test, y_test):.3f}")

# Print feature importance
feature_importance = list(zip(tfidf_vectorizer.get_feature_names_out(), rf_classifier.feature_importances_))
sorted_features = sorted(feature_importance, key=lambda x: x[1], reverse=True)
print("\nTop 10 most important words:")
for word, importance in sorted_features[:10]:
    print(f"{word}: {importance:.4f}")

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   23.3s


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   57.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


Random Forest with TF-IDF Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.82      0.79      6296
           1       0.75      0.68      0.72      4930

    accuracy                           0.76     11226
   macro avg       0.76      0.75      0.75     11226
weighted avg       0.76      0.76      0.76     11226



[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


Random Forest with TF-IDF accuracy: 0.761

Top 10 most important words:
російський: 0.0148
військо: 0.0140
обстріл: 0.0140
окупант: 0.0125
європейський: 0.0109
джерело: 0.0105
військовий: 0.0095
війнути: 0.0085
до: 0.0079
втрата: 0.0079


# Optuna Hyperparameter search

# https://optuna.org/

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
    }
    
    # TF-IDF parameters
    max_features = trial.suggest_int('tfidf_max_features', 500, 3000)
    
    # Create features
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    X = tfidf_vectorizer.fit_transform(train_test['train']['ukr_text'])
    y = np.array(train_test['train']['target'])  # Convert to numpy array
    
    # Create and train model
    rf_classifier = RandomForestClassifier(
        random_state=42,
        **params
    )
    
    # Perform cross-validation
    scores = cross_val_score(rf_classifier, X, y, cv=3, scoring='f1')
    return scores.mean()

# Create and run study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print results
print("Best trial:")
print("  Value:", study.best_trial.value)
print("  Params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

# Train final model with best parameters
best_params = study.best_trial.params

# Split the parameters
tfidf_params = {'max_features': best_params.pop('tfidf_max_features')}

# Create final model with best parameters
final_tfidf = TfidfVectorizer(**tfidf_params)
X_train = final_tfidf.fit_transform(train_test['train']['ukr_text'])
X_test = final_tfidf.transform(train_test['test']['ukr_text'])
y_train = np.array(train_test['train']['target'])
y_test = np.array(train_test['test']['target'])

# Train final model
final_rf = RandomForestClassifier(random_state=42, **best_params)
final_rf.fit(X_train, y_train)

# Evaluate
y_pred = final_rf.predict(X_test)
print("\nFinal Model Performance:")
print(classification_report(y_test, y_pred))

# Print feature importance
feature_importance = list(zip(final_tfidf.get_feature_names_out(), final_rf.feature_importances_))
sorted_features = sorted(feature_importance, key=lambda x: x[1], reverse=True)
print("\nTop 10 most important words:")
for word, importance in sorted_features[:10]:
    print(f"{word}: {importance:.4f}")